In [38]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras import optimizers
import sys
from keras.callbacks import LambdaCallback
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


import numpy as np
import os
import time

In [6]:
tweet_data=pd.read_csv('../Load_Tweets/data/tweet_data.csv')

In [7]:
tweet_text = tweet_data['TEXT']

In [8]:
tweet_text_all = tweet_data['TEXT'].str.cat(sep=' ')

In [9]:
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day. Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN. [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti Earthquake. Unbelieva

In [10]:
print ('Length of text: {} characters'.format(len(tweet_text_all)))

Length of text: 1513769 characters


In [11]:
# Take a look at the first 250 characters in text
print(tweet_text_all[:250])

PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been t


In [12]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [13]:
import re
tweet_text_all=re.sub('(\w)\.(\w)', '\\1. \\2', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4})(\w)', ' \\1 \\2 \\3', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4}) ', '\\1 \\2 ',tweet_text_all) #fdj... fdj ...
tweet_text_all=re.sub(' (\.{2,10})(\w)', ' \\1 \\2',tweet_text_all) #...fdj ... fdj
tweet_text_all=re.sub('[)(]', '', tweet_text_all) #can make paranteses their own token

In [14]:
tweet_text_all=re.sub('!(.)',' ! \\1', tweet_text_all)
tweet_text_all=re.sub('\"', '', tweet_text_all) # can keep the qoutes
tweet_text_all=re.sub("\'", '', tweet_text_all)
tweet_text_all=re.sub("\”", '', tweet_text_all)
tweet_text_all=re.sub("\‘", '', tweet_text_all)
tweet_text_all=re.sub("\’", '', tweet_text_all)
tweet_text_all=re.sub("#(.)", '# \\1', tweet_text_all) 
tweet_text_all=re.sub("@(.)", '@ \\1', tweet_text_all)
tweet_text_all=re.sub("(.),", '\\1 ,', tweet_text_all)
tweet_text_all=re.sub(",(.)", ', \\1', tweet_text_all)
tweet_text_all=re.sub("(.):", '\\1 :', tweet_text_all)


In [15]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [16]:
# tweet_text_all=re.sub('([^.])\.([^.])', '\\1 EOS\\2', tweet_text_all) 
# tweet_text_all = tweet_text_all.replace(".", ' <EOS>')
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest !  Rigged system !  Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West Palm Beach ,  FL at noon :[URL]Cincinnati ,  OH this 7 :30pm :[URL] Wow ,  @ CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD !  Thank you Texas !  If you havent registered to VOTE- today is your last day. Go to : [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at : [URL] and get out to… [URL] DONT LET HER FOOL US AGAIN. [URL] Crookeds State Dept gave special attention to Friends of Bill after the Haiti Earthqua

In [17]:
list_of_words=pd.Series(tweet_text_all.split())
print ('Length of text: {} words'.format(len(list_of_words)))

Length of text: 283502 words


In [18]:
#number of unique words
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

23452 unique words


In [19]:
# The unique characters in the file
vocab = sorted(set(tweet_text_all))
print ('{} unique characters'.format(len(vocab)))

362 unique characters


In [20]:
# #replace apostrophes in dictionary keys 
# for i in range (len(list_of_words)):
#     list_of_words[i]=list_of_words[i].replace("‘", '').replace("’", '').replace("'", '').replace('"', '')

In [21]:
# #see how many words after apostrophe deletion
# word_vocab=sorted(set(list_of_words))
# print ('{} unique words'.format(len(word_vocab)))

In [22]:
word_vocab.append('UNK')

In [23]:
word_vocab[-1]

'UNK'

In [24]:
#create forward and reverse word index
word2idx = dict((c, i) for i, c in enumerate(word_vocab))
# idx2word = dict((i, c) for i, c in enumerate(word_vocab,1 ))
idx2word= np.asarray(word_vocab)

text_as_int = np.array([word2idx[w] for w in list_of_words])

In [25]:
print('{')
for word,_ in zip(word2idx, range(2000)):
    print('  {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print('  ...\n}')

{
  'RepLaMalfa': 8602,
  'kicked': 16903,
  'O' : 7382,
  'definitively': 13918,
  '306-223?': 560,
  'Strength.': 9724,
  'MailOnline': 6537,
  'alone': 11751,
  'LONGING': 6050,
  'Harris': 4943,
  'response.': 19906,
  'Christians': 2640,
  'Commercial': 2807,
  'type': 21919,
  'crowds.': 13697,
  'recite': 19586,
  'Lars': 6106,
  'energizer': 14686,
  'Guaido': 4770,
  'highs': 16093,
  'Resistance': 8664,
  'INDPrimary': 5282,
  'TrumpForPresident': 10389,
  'Seema': 9253,
  'Geneva': 4584,
  'McGurk': 6719,
  'Matthew.': 6677,
  'Students': 9740,
  '1944': 349,
  'atrocious': 12065,
  'tolerate': 21667,
  'Defense': 3348,
  'calls.': 12724,
  'Illinois.': 5351,
  'cherished': 12971,
  'Kinston': 5919,
  'bribery?': 12586,
  'Topeka': 10263,
  'belongs.': 12321,
  'greater': 15776,
  'storm': 21019,
  'Person': 7920,
  'When': 11147,
  'insane': 16573,
  'zulu_out': 22833,
  'CBNNews': 2242,
  'PeteStauber': 7926,
  'please': 18813,
  '☀️☀️☀️Thank': 23372,
  'JeffTutorials': 56

In [26]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- words mapped to int ---- > {}'.format(repr(' '.join(list_of_words[:13])), text_as_int[:13]))

'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest' ---- words mapped to int ---- > [ 7646  9912  7672  7684     3  3025 11376 10783 17240 18735 12696 21484
 14218]


In [27]:
idx2word

array(['!', '!#', '!Full', ..., '🦃', '\U0010fc00', 'UNK'], dtype='<U123')

In [113]:
length_train=int(len(text_as_int)*0.8)
text_as_int_train=text_as_int[: length_train]
text_as_int_val=text_as_int[length_train :]
text_as_int_train.shape
text_as_int_val.shape

(56701,)

In [166]:
# The maximum length sentence we want for a single input in characters
seq_length = 20
examples_per_epoch = len(list_of_words)//seq_length

# Create training examples / targets
word_dataset_train = tf.data.Dataset.from_tensor_slices(text_as_int_train)
word_dataset_val = tf.data.Dataset.from_tensor_slices(text_as_int_val)

for i in word_dataset_train.take(5):
  print(idx2word[i.numpy()])

PAY
TO
PLAY
POLITICS.
#


In [167]:
sequences_train = word_dataset_train.batch(seq_length+1, drop_remainder=True)
sequences_val = word_dataset_val.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2word[item.numpy()])))  


'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So'
'dishonest ! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has been'
'there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before'
'and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West Palm'
'Beach , FL at noon :[URL]Cincinnati , OH this 7 :30pm :[URL] Wow , @ CNN Town Hall questions were given'


In [168]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset_train = sequences_train.map(split_input_target)
dataset_val = sequences_val.map(split_input_target)

In [169]:
for input_example, target_example in  dataset_train.take(10):
  print ('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'PAY TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks.'
Target data: 'TO PLAY POLITICS. # CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So'
Input data:  'dishonest ! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has'
Target data: '! Rigged system ! Crooked Hillary Clinton likes to talk about the things she will do but she has been'
Input data:  'there for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen'
Target data: 'for 30 years - why didnt she do them? Thank you Florida- a MOVEMENT that has never been seen before'
Input data:  'and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West'
Target data: 'will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio ! West 

In [170]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

Step    0
  input: 275 ('13')
  expected output: 20955 ('states')
Step    1
  input: 20955 ('states')
  expected output: 15957 ('have')
Step    2
  input: 15957 ('have')
  expected output: 22309 ('voter')
Step    3
  input: 22309 ('voter')
  expected output: 19690 ('registration')
Step    4
  input: 19690 ('registration')
  expected output: 13800 ('deadlines')


In [171]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset_val=dataset_val = dataset_val.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


dataset_val

<DatasetV1Adapter shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [172]:
dataset_train

<DatasetV1Adapter shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [173]:
embeddings_index = {}
f = open('../word_embeding/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [200]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)+1

# The embedding dimension 
embedding_dim = 25

# Number of RNN units
rnn_units = 1024
embedding_matrix = np.zeros((vocab_size, embedding_dim)) 

for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [201]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  print ("on the GPU BABY!")
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

on the GPU BABY!


In [202]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                batch_input_shape=[batch_size, None,],
                                trainable=False),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

In [203]:
model = build_model(
  vocab_size = vocab_size, 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [204]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 25)            586350    
_________________________________________________________________
cu_dnngru_6 (CuDNNGRU)       (64, None, 1024)          3228672   
_________________________________________________________________
dense_6 (Dense)              (64, None, 23454)         24040350  
Total params: 27,855,372
Trainable params: 27,269,022
Non-trainable params: 586,350
_________________________________________________________________


In [205]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 20, 23454) # (batch_size, sequence_length, vocab_size)


In [206]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [207]:
sampled_indices

array([20826,  3800, 12965, 12220, 14706,  9759,  2213,  1742,  3633,
       23380,  6369,  8432, 10727,  6957, 22364,  5058,   540,  6497,
       16965,  9278])

In [208]:
print("Input: \n", repr(" ".join(idx2word[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

Input: 
 'Haitians. Probably should record future meetings - unfortunately , no trust ! Sadly , Democrats want to stop paying our'

Next Char Predictions: 
 'spectacular Election cheer… basement enjoy Suffolk_Sheriff C. BILLION/yr. Ducey ⚠️If Lá Randy VOTE… MoskowitzEva wages- Highs 3%Christie MSNBC lacks SenRobPortman'


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [209]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 20, 23454)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.062788


In [210]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [211]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)


checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)


In [214]:
history = model.fit(
    dataset_train.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, validation_data=dataset_val, validation_steps=30, callbacks=[checkpoint_callback, early_stopping])

Epoch 1/10
221/221 [==============================] - 33s 147ms/step - loss: 7.5984 - val_loss: 8.1384
Epoch 2/10
 43/221 [====>.........................] - ETA: 21s - loss: 7.5280

KeyboardInterrupt: 

In [189]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [190]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [191]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 25)             586350    
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       (1, None, 1024)           3228672   
_________________________________________________________________
dense_5 (Dense)              (1, None, 23454)          24040350  
Total params: 27,855,372
Trainable params: 27,269,022
Non-trainable params: 586,350
_________________________________________________________________


In [68]:
model1 = load_model('../Saved_models/embedding_2_1.h5')

In [69]:
model1.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss =loss)

In [192]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of words to generate
  num_generate = 30

  # Converting our start string to numbers (vectorizing) 
  if start_string!='':
      for word in start_string.split():
        if word in word2idx:
          input_eval = [word2idx[word]]
          input_eval = tf.expand_dims(input_eval, 0)
          print (input_eval)
        else:
            input_eval= [word2idx['UNK']]
            input_eval = tf.expand_dims(input_eval, 0)
  else: 
    input_eval= [word2idx['UNK']]
    input_eval = tf.expand_dims(input_eval, 0)

    

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2word[predicted_id])

  return (start_string + ' ' + ' '.join(text_generated))

In [193]:
print(generate_text(model1, start_string=u"so what"))

tf.Tensor([[20713]], shape=(1, 1), dtype=int32)
tf.Tensor([[22522]], shape=(1, 1), dtype=int32)
so what guards happen for. , was is of little small-business in out of Puerto Rico Today is have the as election .... @ so was a big one one , they


In [81]:
model.save('../Saved_models/embedding_2_1.h5')

In [92]:
idx2word[20713]

'so'